# K-Means

In [1]:
from typing import Union, Literal

import numpy as np
import pandas as pd

from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from torchvision import datasets

In [2]:
class KMeans:
    """
    Perform KMeans clustering on a dataset.
    """

    def __init__(self,
                 algorithm : Literal['lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan'] = 'lloyd',
                 init : Literal['random', 'random-data', 'kmeans++', 'greedy'] = 'random',
                 seed : Union[int, None] = None):
        """
        Initialize the KMeans object.

        Parameters
        ----------
        algorithm : {'lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan', 'extended-hartigan-2', 'safe-hartigan-2', 'hartigan-2'}
            Algorithm to use. Either 'lloyd' or 'extended-hartigan' or 'safe-hartigan' or 'hartigan' or 'extended-hartigan-2'

        init : {'random', 'random-data', 'kmeans++', 'greedy'}
            Initialization method. Either 'random' or 'random-data' or 'kmeans++' or 'greedy'

        seed : int
            Seed for random generator
        """

        assert algorithm in ['lloyd', 'extended-hartigan', 'safe-hartigan', 'hartigan'], "algorithm must be either 'lloyd', 'extended-hartigan', 'safe-hartigan' or 'hartigan'"
        assert init in ['random', 'random-data', 'kmeans++', 'greedy'], "init must be either 'random', 'random-data', 'kmeans++' or 'greedy'"
        assert seed is None or isinstance(seed, int), "seed must be an int or None"

        self.algorithm = algorithm
        self.init = init
        self.seed = seed

        self.data = None
        self.k = None
        self.centroids = None
        self.y_pred = None


    def fit(self, data : np.ndarray, k : int, debug=False):
        """
        Fit the model to the data.

        Parameters
        ----------
        data : np.ndarray
            nxd DataFrame of n samples with d features
        k : int
            Number of clusters

        Returns
        -------
        np.ndarray
            Array of shape (k, d) with cluster centroids
        np.ndarray
            Array of length n with cluster assignments for each sample
        """

        assert isinstance(data, np.ndarray), "data must be a numpy array"
        assert len(data.shape) == 2, "data must be a 2D array"
        assert isinstance(k, int), "k must be an int"
        assert 0 < k <= len(data), "k must be at least 0 and at most the number of samples"
        assert isinstance(debug, bool), "debug must be a boolean"

        self.data = data
        self.k = k

        np.random.seed(self.seed)

        # initialize centroids
        self._init_centroids(debug)
        debug and print('initial centroids:\n', self.centroids)

        ## TODO: implement clustering algorithm

        if self.algorithm == 'lloyd':
            self._lloyd(debug)
        elif self.algorithm == 'extended-hartigan':
            self._extended_hartigan(always_safe=False, debug=debug)
        elif self.algorithm == 'safe-hartigan':
            self._extended_hartigan(always_safe=True, debug=debug)
        elif self.algorithm == 'hartigan':
            return NotImplemented
            self._hartigan(debug)
        
        print('final centroids:\n', self.centroids)
        print('final y_pred:', self.y_pred)


    def _init_centroids(self, debug=False):
        """
        Initialize the centroids.
        """

        if self.init == 'random':

            # choose k random data points as initial centroids
            idx = np.random.choice(self.data.shape[0], self.k, replace=False)
            self.centroids = self.data[idx]

        elif self.init == 'random-data':

            # assign each data point to a random cluster
            clusters = np.random.choice(self.k, self.data.shape[0])

            # check that at least one point is assigned to each cluster
            while len(set(clusters)) < self.k:
                clusters = np.random.choice(self.k, self.data.shape[0])
            self.y_pred = clusters
            self.centroids = self._move_centroids(debug)

        elif self.init == 'kmeans++':

            # choose first centroid randomly
            centroids = np.zeros((self.k, self.data.shape[1]))
            centroids[0] = self.data[np.random.choice(self.data.shape[0], 1, replace=False)[0]]
            debug and print('centroids:\n', centroids)

            # iterate over remaining k-1 centroids
            for i in range(1, self.k):
                debug and print('iteration', i)

                # calculate distance squared of each point to closest centroid
                dist = np.array([min([np.linalg.norm(c-x)**2 for c in centroids[:i]]) for x in self.data])

                # probabilities are given by the normalized distance squared
                probs = dist / dist.sum()
                debug and print('probs:', probs)

                # cumulate probabilities
                cumprobs = probs.cumsum()

                # choose next centroid randomly based on cumulated probabilities
                r = np.random.rand()
                debug and print('r:', r)
                for j, p in enumerate(cumprobs):
                    if r < p:
                        break

                centroids[i] = self.data[j]
                debug and print('centroids:\n', centroids)

            self.centroids = centroids

        elif self.init == 'greedy':
            return NotImplemented


    def _lloyd(self, debug=False):
        """
        Lloyd's algorithm for k-means clustering.
        """

        debug and print('\nRunning Lloyd\'s algorithm...')

        while True:
            
            debug and print('New iteration')

            # assign each data point to the closest centroid
            self.y_pred = self._assign_clusters(debug)
            debug and print('y_pred:', self.y_pred)

            # move centroids to the mean of their cluster
            new_centroids = self._move_centroids(debug)

            # check for convergence
            if np.allclose(self.centroids, new_centroids):
                break

            self.centroids = new_centroids


    def _extended_hartigan(self, always_safe=False, debug=False):
        """
        Extended Hartigan algorithm for k-means clustering.
        """

        debug and print('\nRunning Extended Hartigan algorithm...')

        # first assignment
        if self.y_pred is None:
            self.y_pred = self._assign_clusters(debug)

        while True:
            # start with unsafe mode    
            safe_mode = False

            # create an empty dictionary of new candidates
            candidates = {}

            for datapoint_id in range(len(self.data)):
                debug and print('\ndatapoint_id:', datapoint_id)

                # calculate cost of current assignment which remains invariant
                current_centroid_id = self.y_pred[datapoint_id]
                cluster_size = np.where(self.y_pred == current_centroid_id)[0].shape[0]
                prefactor = cluster_size / (cluster_size - 1) if cluster_size > 1 else 0
                
                current_cost = prefactor * np.linalg.norm(self.data[datapoint_id] - self.centroids[current_centroid_id])**2
                debug and print('current_cost:', current_cost)

                # if current_cost is 0, delta_cost will always be positive
                if current_cost == 0:
                    continue

                # iterate only on possible new centroid assignments
                for centroid_id in np.setdiff1d(self.y_pred, current_centroid_id):
                    delta_cost = self._delta_cost(current_cost, datapoint_id, centroid_id)
                    debug and print(f'delta_cost for datapoint {datapoint_id} from centroid {current_centroid_id} to centroid {centroid_id}:', delta_cost)

                    # datapoint is a candidate if it reduces the cost
                    # if more reassignments reduce the cost, the best one is stored (the one producing the most negfative delta_cost)
                    if delta_cost < 0 and (candidates.get(datapoint_id) is None or delta_cost < candidates[datapoint_id][0]):
                        candidates[datapoint_id] = [delta_cost, current_centroid_id, centroid_id]

            debug and print('\ncandidates:', candidates)
            
            # break at convergence
            if not candidates:      ## [] -> False
                debug and print('no more candidates')
                break    

            # proceed in unsafe mode
            if not safe_mode and not always_safe:
                debug and print('\nentered in UNSAFE mode')

                # store current state for possible rollback
                rollback = self.y_pred.copy()

                # calculate original cost
                debug and print('\ncalculating original cost...')
                original_cost = self._tot_cluster_cost(self.centroids, self.y_pred, debug=debug)
                debug and print('original_cost:', original_cost)

                # accept all candidates
                for candidate in candidates.keys():
                    debug and print('candidate:', candidate)
                    
                    [delta_cost, current_centroid_id, new_centroid_id] = candidates[candidate]
                    debug and print('y_pred before:', self.y_pred)

                    # update closest_points_ids assigning datapoint to new_centroid_id
                    self.y_pred[candidate] = new_centroid_id
                    debug and print('y_pred after:', self.y_pred)

                new_centroids = self._move_centroids(debug)

                debug and print('\ncalculating new cost...')
                if self._tot_cluster_cost(new_centroids, self.y_pred, debug=debug) >= original_cost:
                    
                    # new clustering is more expensive, proceed in safe mode
                    safe_mode = True
                    self.y_pred = rollback

            # start new condition since safe mode can be entered from unsafe mode
            if safe_mode or always_safe:
                debug and print('\nentered in SAFE mode')

                unchanged_clusters = list(range(self.k))
                for _, [delta_cost, current_centroid_id, new_centroid_id] in sorted(candidates.items(), key=lambda e: e[1][1]):

                    # if both clusters are still unchanged, accept the candidate
                    if current_centroid_id in unchanged_clusters and new_centroid_id in unchanged_clusters:
                        self.y_pred[datapoint_id] = new_centroid_id
                        unchanged_clusters.remove(current_centroid_id)
                        unchanged_clusters.remove(new_centroid_id)

                    # if we cannot operate on any more clusters, break
                    if not unchanged_clusters:
                        break

                new_centroids = self._move_centroids(debug)
        
            self.centroids = new_centroids

        return NotImplemented


    def _move_centroids(self, debug=False):
        """
        Move the centroids to the mean of their cluster.
        """

        debug and print('\n  moving centroids...')
        debug and print('  | y_pred:', self.y_pred)
        debug and print('  | data:\n', self.data)
        debug and print('  | centroids_before:\n', self.centroids)

        centroids = np.zeros((self.k, self.data.shape[1]))
        for centroid_id in range(self.k):
            cluster_points = self.data[self.y_pred == centroid_id]
            
            # if centroid has no points assigned to it, reassign it randomly
            if len(cluster_points) == 0:
                debug and print(f"  Centroid {centroid_id} is empty. Reassigning.")
                centroids[centroid_id] = self.data[np.random.choice(len(self.data))]
            else:
                centroids[centroid_id] = np.mean(cluster_points, axis=0)

        debug and print('  centroids_after:\n', centroids)

        return centroids


    def _assign_clusters(self, debug=False):
        """
        Assign each data point to the closest centroid.
        """

        y_pred = np.zeros(len(self.data), dtype=int)
        for i, x in enumerate(self.data):
            debug and print(f'{i}:', [np.linalg.norm(x-c)**2 for c in self.centroids])
            y_pred[i] = np.argmin([np.linalg.norm(x-c)**2 for c in self.centroids])

        debug and print('y_pred:', y_pred)

        return y_pred


    def _delta_cost(self, cost, datapoint_id, centroid_id):
        """
        Compute the change in cost if datapoint is reassigned to centroid_id
        """

        cluster_size = np.where(self.y_pred == centroid_id)[0].shape[0]
        prefactor = cluster_size / (cluster_size + 1)

        # cost of new assignment
        new_cost = prefactor * np.linalg.norm(self.data[datapoint_id] - self.centroids[centroid_id])**2

        return new_cost - cost


    def _tot_cluster_cost(self, centroids, points_ids, debug=False):
        """
        Compute the overall cost of clustering
        """
        
        debug and print('  calculating _tot_cluster_cost')
        
        partial_sum = []
        for centroid_id in range(centroids.shape[0]):
            cluster_items = np.where(points_ids == centroid_id)[0]
            partial_sum.append(np.sum(np.square(self.data[cluster_items] - self.centroids[centroid_id])))

            debug and print('  | centroid_id:', centroid_id)
            debug and print('  | centroid:', centroids[centroid_id])
            debug and print('  | cluster_items:', cluster_items)
            debug and print('  | partial_sum:', partial_sum)
        
        debug and print('  _tot_cluster_cost:', np.sum(partial_sum))

        return np.sum(partial_sum)


In [3]:
def accuracy(y_true : np.ndarray, y_pred : np.ndarray):
    """
    Compute the accuracy of the clustering.
    
    Parameters
    ----------
    y_true : np.ndarray
        True labels of the samples
    y_pred : np.ndarray
        Predicted labels of the samples
    
    Returns
    -------
    float
        Accuracy of the clustering through Hungarian algorithm
    """
    
    assert isinstance(y_true, np.ndarray), "y_true must be a numpy array"
    assert isinstance(y_pred, np.ndarray), "y_pred must be a numpy array"

    # create C matrix
    n_classes = max(max(y_true), max(y_pred)) + 1
    C = np.zeros((n_classes, n_classes), dtype=int)
    for true_label, pred_label in zip(y_true, y_pred):
        C[true_label, pred_label] += 1
    
    # Solve assignment problem
    row_ind, col_ind = linear_sum_assignment(-C)
    
    # Calculate accuracy
    matched = C[row_ind, col_ind].sum(axis=0)
    accuracy = matched / len(y_true)
    return accuracy

In [98]:
# debug
a = np.array([[1, 2], [3, 4], [11, 12], [13, 14], [19, 20]])

kmeans = KMeans(algorithm='extended-hartigan', init='random')
kmeans.fit(a, 3, True)
accuracy(np.array([0, 0, 1, 1, 2]), kmeans.y_pred)

initial centroids:
 [[19 20]
 [ 1  2]
 [11 12]]

Running Extended Hartigan algorithm...
0: [np.float64(648.0), np.float64(0.0), np.float64(200.00000000000003)]
1: [np.float64(512.0000000000001), np.float64(8.000000000000002), np.float64(128.00000000000003)]
2: [np.float64(128.00000000000003), np.float64(200.00000000000003), np.float64(0.0)]
3: [np.float64(71.99999999999999), np.float64(287.99999999999994), np.float64(8.000000000000002)]
4: [np.float64(0.0), np.float64(648.0), np.float64(128.00000000000003)]
y_pred: [1 1 2 2 0]

datapoint_id: 0
current_cost: 0.0

datapoint_id: 1
current_cost: 16.000000000000004
delta_cost for datapoint 1 from centroid 1 to centroid 0: 240.00000000000006
delta_cost for datapoint 1 from centroid 1 to centroid 2: 69.33333333333334

datapoint_id: 2
current_cost: 0.0

datapoint_id: 3
current_cost: 16.000000000000004
delta_cost for datapoint 3 from centroid 2 to centroid 0: 19.99999999999999
delta_cost for datapoint 3 from centroid 2 to centroid 1: 175.999999

np.float64(1.0)

In [226]:
a = np.array([[1, 2], [3, 4], [11, 12], [13, 14], [19, 20]])

kmeans = KMeans(algorithm='lloyd', init='kmeans++')
kmeans.fit(a, 3, True)
accuracy(np.array([0, 0, 1, 1, 2]), kmeans.y_pred)

centroids:
 [[1. 2.]
 [0. 0.]
 [0. 0.]]
iteration 1
probs: [0.         0.00699301 0.17482517 0.25174825 0.56643357]
r: 0.11165717709251399
centroids:
 [[ 1.  2.]
 [11. 12.]
 [ 0.  0.]]
iteration 2
probs: [0.         0.05555556 0.         0.05555556 0.88888889]
r: 0.9960458970299656
centroids:
 [[ 1.  2.]
 [11. 12.]
 [19. 20.]]
initial centroids:
 [[ 1.  2.]
 [11. 12.]
 [19. 20.]]
Running Lloyd's algorithm...
New iteration
[np.float64(0.0), np.float64(200.00000000000003), np.float64(648.0)]
[np.float64(8.000000000000002), np.float64(128.00000000000003), np.float64(512.0000000000001)]
[np.float64(200.00000000000003), np.float64(0.0), np.float64(128.00000000000003)]
[np.float64(287.99999999999994), np.float64(8.000000000000002), np.float64(71.99999999999999)]
[np.float64(648.0), np.float64(128.00000000000003), np.float64(0.0)]
y_pred: [0 0 1 1 2]
y_pred: [0 0 1 1 2]
data:
 [[ 1  2]
 [ 3  4]
 [11 12]
 [13 14]
 [19 20]]
centroids_before:
 [[ 1.  2.]
 [11. 12.]
 [19. 20.]]
centroids_after:
 

np.float64(1.0)

In [381]:
a = np.array([[1, 2], [3, 4], [11, 12], [13, 14], [19, 20]])

kmeans = KMeans(algorithm='extended-hartigan', init='random')
kmeans.fit(a, 3, True)
accuracy(np.array([0, 0, 1, 1, 2]), kmeans.y_pred)

initial centroids:
 [[13 14]
 [ 3  4]
 [19 20]]

Running Extended Hartigan algorithm...
0: [np.float64(287.99999999999994), np.float64(8.000000000000002), np.float64(648.0)]
1: [np.float64(200.00000000000003), np.float64(0.0), np.float64(512.0000000000001)]
2: [np.float64(8.000000000000002), np.float64(128.00000000000003), np.float64(128.00000000000003)]
3: [np.float64(0.0), np.float64(200.00000000000003), np.float64(71.99999999999999)]
4: [np.float64(71.99999999999999), np.float64(512.0000000000001), np.float64(0.0)]

datapoint_id: 0
current_cost: 16.000000000000004
delta_cost for datapoint 0 from centroid 1 to centroid 0: 175.99999999999994
delta_cost for datapoint 0 from centroid 1 to centroid 2: 308.0

datapoint_id: 1
current_cost: 0.0
delta_cost for datapoint 1 from centroid 1 to centroid 0: 133.33333333333334
delta_cost for datapoint 1 from centroid 1 to centroid 2: 256.00000000000006

datapoint_id: 2
current_cost: 16.000000000000004
delta_cost for datapoint 2 from centroid 0 to 

np.float64(1.0)

In [393]:
a = np.array([[1, 2], [3, 4], [11, 12], [13, 14], [19, 20]])

kmeans = KMeans(algorithm='safe-hartigan', init='random')
kmeans.fit(a, 3, True)
accuracy(np.array([0, 0, 1, 1, 2]), kmeans.y_pred)

initial centroids:
 [[19 20]
 [11 12]
 [ 1  2]]

Running Extended Hartigan algorithm...
0: [np.float64(648.0), np.float64(200.00000000000003), np.float64(0.0)]
1: [np.float64(512.0000000000001), np.float64(128.00000000000003), np.float64(8.000000000000002)]
2: [np.float64(128.00000000000003), np.float64(0.0), np.float64(200.00000000000003)]
3: [np.float64(71.99999999999999), np.float64(8.000000000000002), np.float64(287.99999999999994)]
4: [np.float64(0.0), np.float64(128.00000000000003), np.float64(648.0)]

datapoint_id: 0
current_cost: 0.0
delta_cost for datapoint 0 from centroid 2 to centroid 0: 324.0
delta_cost for datapoint 0 from centroid 2 to centroid 1: 133.33333333333334

datapoint_id: 1
current_cost: 16.000000000000004
delta_cost for datapoint 1 from centroid 2 to centroid 0: 240.00000000000006
delta_cost for datapoint 1 from centroid 2 to centroid 1: 69.33333333333334

datapoint_id: 2
current_cost: 0.0
delta_cost for datapoint 2 from centroid 1 to centroid 0: 64.000000000000

np.float64(1.0)

In [402]:
mnist = datasets.MNIST('data', train=True, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.37MB/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 278kB/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 2.50MB/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [442]:
data = mnist.data.numpy().reshape(-1, 28*28)
kmeans = KMeans(algorithm='extended-hartigan', init='random')
kmeans.fit(data[:20], 10)
accuracy(mnist.targets.numpy()[:5000], kmeans.y_pred)

KeyboardInterrupt: 